# Lab 1, Question 8

In [ ]:
# Install the geemap package (only needs to be run once, uncomment below and run it the first time you run this notebook in a session).
# !pip install geemap

In [ ]:
if 'google.colab' in str(get_ipython()):
    from google.colab import userdata
    EE_PROJECT_ID = userdata.get('EE_PROJECT_ID') 
else:
    from dotenv import load_dotenv
    import os
    load_dotenv()  # take environment variables
    EE_PROJECT_ID = os.getenv('EE_PROJECT_ID')

In [ ]:
# Set up GEE API
import ee
ee.Authenticate()
ee.Initialize(project=EE_PROJECT_ID)
import geemap

### (8) Using Sentinel 2, create a figure that does the following over the city of Christchurch, NZ:
*   Conduct a basic quality assessment of the data you use, using a cloudy pixel % estimate.
* Describe/show the spectral reflectance signatures of: urban, forest, grassland, water and bare soil.
* Illustrates the sample areas that you have used to create these spectral signatures.

Provide a publication quality copy of this figure in the answer proforma. Publication quality means that:  
1.   It is at a good quality (resolution) and not blurry.
2.   The font size of all elements is big enough to read easily on a screen when placed in the document.
3. Has a figure caption that ensures the reader knows what you have done and why.
4. Tells the story 'at a glance'. Don't make your viewer work too hard to understand your core message.

(25 pts)




In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import urllib.request
import tempfile
import matplotlib.image as mpimg
import numpy as np

### Step 1: Cloud Masking using QA60

Define Point expanded ROI

In [ ]:
# St. Albans Park Park
christchurch_coordinates = [172.6450, -43.5096]

roi_point = ee.Geometry.Point(christchurch_coordinates)

# Define a region for fetching the image thumbnail
expanded_roi = roi_point.buffer(7000).bounds()  # box for image preview

Load Image with least cloud in the year of 2024

In [ ]:
# # Load Sentinel-2 L2A collection for a summer month
# # Using a recent summer for good vegetation and sun angle
s2_collection = 'COPERNICUS/S2_SR_HARMONIZED'
from_date = '2024-01-01'
to_date = '2024-12-31'

# # Test masked map on more cloudy day
# roi_cloud_img = ee.ImageCollection(s2_collection) \
#     .filterBounds(roi_point) \
#     .filterDate(from_date, to_date) \
#     .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
#     .sort('CLOUDY_PIXEL_PERCENTAGE', ascending=False) \
#     .first()

roi_cloud_img = ee.ImageCollection(s2_collection) \
    .filterBounds(roi_point) \
    .filterDate(from_date, to_date) \
    .sort('CLOUDY_PIXEL_PERCENTAGE') \
    .first()

print('Image ID:', roi_cloud_img.get('PRODUCT_ID').getInfo())
print('Cloud Cover (whole scene):', roi_cloud_img.get('CLOUDY_PIXEL_PERCENTAGE').getInfo())
print('Over Land Cloud Cover (whole scene):', roi_cloud_img.get('CLOUDY_PIXEL_OVER_LAND_PERCENTAGE').getInfo())

Cloud masking

In [ ]:
# https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED
def mask_s2_clouds(image):
    # QA60 band is Cloud mask
    qa = image.select('QA60')

    # Bitmask
    #   Bit 10: Opaque clouds
    #     0: No opaque clouds
    #     1: Opaque clouds present
    #   Bit 11: Cirrus clouds
    #     0: No cirrus clouds
    #     1: Cirrus clouds present
    mask = qa.bitwiseAnd(1 << 10).eq(0) \
        .And(qa.bitwiseAnd(1 << 11).eq(0))

    # The assets contain 12 UINT16 spectral bands representing SR scaled by 10000
    return image.updateMask(mask).divide(10000)

# Apply the cloud mask
roi_cloud_masked_img = mask_s2_clouds(roi_cloud_img)

Show the map to have a look

In [ ]:
vis_params_true_color = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3000,
    'gamma': 1.4
}
vis_params_masked = {**vis_params_true_color, 'max': 0.3}

roi_map = geemap.Map(center=christchurch_coordinates[::-1], zoom=12)
roi_map.addLayer(roi_cloud_img, vis_params_true_color, 'True Colour Image (Cloudy)')
roi_map.addLayer(roi_cloud_masked_img, vis_params_masked, 'Cloud Masked')
roi_map

## Step 2: he spectral reflectance signatures
Type of surfaces: urban, forest, grassland, water and bare soil.

`get_reflectances` from Sentinent-2 bands and `get_region_coords` function
 

In [ ]:
# sentinels.copernicus.eu/documents/247904/685211/Sentinel-2_User_Handbook
s2_band_names = {
    'B1': '443.9 - Aerosols', # 60 meters	443.9nm (S2A) / 442.3nm (S2B)
    'B2': '496.6 - Blue', # 10 meters	496.6nm (S2A) / 492.1nm (S2B)
    'B3': '560 - Green', # 10 meters	560nm (S2A) / 559nm (S2B)
    'B4': '664.5 - Red', # 10 meters	664.5nm (S2A) / 665nm (S2B)
    'B5': '703.9 - Red Edge 1', # 20 meters	703.9nm (S2A) / 703.8nm (S2B)
    'B6': '740.2 - Red Edge 2', # 20 meters	740.2nm (S2A) / 739.1nm (S2B)
    'B7': '782.5 - Red Edge 3', # 20 meters	782.5nm (S2A) / 779.7nm (S2B)
    'B8': '835.1 - NIR', # 10 meters	835.1nm (S2A) / 833nm (S2B)
    'B8A': '864.8 - Red Edge 4', # 20 meters	864.8nm (S2A) / 864nm (S2B)
    'B9': '945 - Water vapor', # 60 meters	945nm (S2A) / 943.2nm (S2B)
    'B11': '1613.7 - SWIR 1', # 20 meters	1613.7nm (S2A) / 1610.4nm (S2B)
    'B12': '2202.4 - SWIR 2', # 20 meters	2202.4nm (S2A) / 2185.7nm (S2B)
}

# Get coordinates of regions
def get_region_coords(region):
    coords = region.coordinates().getInfo()[0]
    lons = [pt[0] for pt in coords]
    lats = [pt[1] for pt in coords]
    return min(lons), max(lons), min(lats), max(lats)

def get_reflectances(region, from_date, to_date):
    # Load surface reflectance image
    image = ee.ImageCollection(s2_collection) \
        .filterBounds(region) \
        .filterDate(from_date, to_date) \
        .sort('CLOUD_COVER') \
        .first()

    # Define bands and scaling
    bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7',
            'B8', 'B8A', 'B9', 'B11', 'B12']
    scale_factor = 0.0001

    # Calculate mean reflectance
    mean_dict = image.select(bands).reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=region,
        scale=10,
        maxPixels=1e9).getInfo()

    return [mean_dict[b] * scale_factor for b in bands]

Define region of interested and surface types areas

In [ ]:
from IPython.display import Image
# Get RGB thumbnail of Christchurch
def thumb_params(region):
    return {
        'region': region,
        'format': 'png',
        'bands': ['B4', 'B3', 'B2'],
        'min': 0,
        'max': 3000,
        'gamma': 1
    }
roi_url = roi_cloud_img.getThumbURL(thumb_params(expanded_roi))

Image(url=roi_url)

In [ ]:
urban = [-43.538735, 172.636655]
forest = [-43.465786, 172.693103]
grassland = [-43.522425, 172.715741]
water = [-43.539055, 172.717482]
soil = [-43.451394, 172.687933]

coordinates = [urban, forest, grassland, water, soil]
surfaces = ['urban', 'forest', 'grassland', 'water', 'soil']

rois = {}

for idx, surface in enumerate(surfaces):
  point = ee.Geometry.Point(coordinates[idx][::-1])
  region = point.buffer(150).bounds()
  
  # Get thumbnail URL and download image data
  surface_url = roi_cloud_img.getThumbURL(thumb_params(region))
  with tempfile.NamedTemporaryFile(suffix=".png") as f:
      urllib.request.urlretrieve(surface_url, f.name)
      img_data = mpimg.imread(f.name)
  
  rois[surface] = {
      'region': region,
      'reflectance': get_reflectances(region, from_date, to_date),
      'coords': get_region_coords(region),
      'img_data': img_data,  # Store actual image data instead of Image object
  }

## Step 3: Ploting

Combine all rois on map and their reflectances plot

In [ ]:
# Ploting all rois
layout = [['A', 'A', 'A', 'A', 'A'],
          ['B', 'C', 'D', 'E', 'F'],
          ['G', 'G', 'G', 'G', 'G']]
fig, axes = plt.subplot_mosaic(layout, figsize=(7, 12), height_ratios=[4,1,4])

surface_loc = ['B', 'C', 'D', 'E', 'F']

ledgend_labels = list(map(lambda s: s.capitalize(), surfaces))

colors = ['#311B92', '#1B5E20', '#FFAB00', '#2962FF', '#DD2C00']
markers = ['.', 'd', '*', 's', 'v']
lines = [
    (0, (1, 1)),
    (0, (5, 5)),
    'solid',
    (0, (5, 1)),
    (0, (3, 1, 1, 1))]

def plot_surface(surface, axe):
    # Display image without extent to fill the entire subplot
    axe.imshow(rois[surface]['img_data'])
    axe.set_title(surface.capitalize(), y=-0.3, fontsize=10)
    axe.set_xticks([])
    axe.set_yticks([])

    # Add colored border around the image
    for spine in axe.spines.values():
        spine.set_edgecolor(colors[surfaces.index(surface)])
        spine.set_linewidth(2)

# Load and plot image with annotation
with tempfile.NamedTemporaryFile(suffix=".png") as f:
    xmin, xmax, ymin, ymax = get_region_coords(expanded_roi)
    urllib.request.urlretrieve(roi_url, f.name)
    img = mpimg.imread(f.name)
axes['A'].imshow(img, extent=[xmin, xmax, ymin, ymax])

# Get the date from the image's metadata
captured_date = ee.Date(
        roi_cloud_img.get('system:time_start')
    ).format('YYYY-MM-dd HH:mm:ss').getInfo()

axes['A'].set_title(
    f"Sentinel-2 Image with mask interested areas\n(captured at {captured_date})")
axes['A'].set_xlim(xmin, xmax)
axes['A'].set_ylim(ymin, ymax)

# Add red rectangle for region of interest
for idx, surface in enumerate(surfaces):
    rxmin, rxmax, rymin, rymax = rois[surface]['coords']
    width = rxmax - rxmin
    height = rymax - rymin
    rect = patches.Rectangle((rxmin, rymin), width, height,
                            linewidth=2, edgecolor=colors[idx], facecolor='none')
    axes['A'].add_patch(rect)
    axes['A'].set_xlabel("Longitude", fontweight='bold')
    axes['A'].tick_params(axis='x', labelrotation=45, labelsize=8)
    axes['A'].tick_params(axis='y', labelsize=8)
    axes['A'].set_ylabel("Latitude", fontweight='bold')
    
    # Add rectangles and images for each surface
    plot_surface(surface, axes[surface_loc[idx]])
    
# Add legend to the first plot
axes['A'].legend(ledgend_labels)

# Plot reflectance
s2_labels = list(s2_band_names.values())
for idx, surface in enumerate(surfaces):
    region = rois[surface]['region']
    reflectance = rois[surface]['reflectance']
    axes['G'].plot(s2_labels, reflectance,
                   color=colors[idx], linestyle=lines[idx], marker='.')
axes['G'].set_ylabel("Reflectance", fontweight='bold')
axes['G'].set_xlabel("Wavelength (nm)", fontweight='bold')
axes['G'].grid(True)
axes['G'].set_title("Spectral Reflectance Curves")
axes['G'].set_xticks(range(len(s2_labels)))
axes['G'].set_xticklabels(s2_labels, rotation=45, ha='right')
axes['G'].legend(ledgend_labels)

fig.suptitle("Image Analysis for Christchurch", fontweight='bold', y=1.01)
plt.tight_layout()
plt.show()

### Extra steps
To make sure bare soil area choice in the Christchurch center

In [ ]:
# Refs:
# https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel-2/barren_soil/
# https://developers.google.com/earth-engine/apidocs/ee-image-expression

# BSI = ((SWIR2 + R)−(NIR + B)) / ((SWIR2 + R)+(NIR + B))
# B11 = SWIR 1, B4 = Red, B8 = NIR, B2 = Blue
# BSI = ((B11 + B4) - (B8 + B2)) / ((B11 + B4) + (B8 + B2))
# This index highlights bare soil areas. Higher values correspond to bare soil.


# remember to use band name as a string `'B11'` with `b()`` function
bsi_exp = "(b('B11') + b('B4') - b('B8') - b('B2')) / (b('B11') + b('B4') + b('B8') + b('B2'))"
bsi = roi_cloud_img.expression(
    bsi_exp).rename('BSI')

# Visualization for the BSI layer.
# Low BSI (such asvegetation) will be green, high BSI (bare soil) will be brown/tan.
bsi_viz_params = {
    'min': -0.1,
    'max': 0.4,
    'palette': ['#228B22', '#FFFFE0', '#D2B48C', '#A0522D']  # ForestGreen, LightYellow, Tan, Sienna
}

# Visualization for a true-color composite (like human eyes)
true_color_viz = {
    'bands': ['B4', 'B3', 'B2'], # Red, Green, Blue
    'min': 0,
    'max': 3000,
    'gamma': 1.4
}

# Visualization for a false-color composite (SWIR)
# Bare soil and built-up areas appear brown/tan, vegetation is green.
swir_viz = {
    'bands': ['B12', 'B8A', 'B4'], # SWIR2, Narrow NIR, Red
    'min': 0,
    'max': 4000,
    'gamma': 1.4
}

bsi_m = geemap.Map(center=christchurch_coordinates[::-1], zoom=12)

# Add the different image layers to the map
bsi_m.addLayer(roi_cloud_img, true_color_viz, 'True Color (RGB)')
bsi_m.addLayer(roi_cloud_img, swir_viz, 'False Color (SWIR)')
bsi_m.addLayer(bsi, bsi_viz_params, 'Bare Soil Index (BSI)')

bsi_m